# **Flooding in Italy - working code (R)**

In [1]:
source("../methods_extRemes.r")

prep_window <- function(rc = c(1,1), w = 4, h = 4) { options(repr.plot.width = rc[2]*w, repr.plot.height = rc[1]*h, repr.plot.res = 200); par(mfrow = rc, pch = 20) }
load_ts <- function(fnm, col.names) { read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = col.names) }

In [2]:
# support function to give required results
mdl_ests <- function(mdl, cov, cov_cf, ev, rp = NA) {

    pars <- mdl$results$par
    disp <- unname(pars["sigma0"] / pars["mu0"])

    if(is.na(rp)) rp <- return_period(mdl, value = ev, covariate = cov, lower = F)
    if(is.finite(rp)) {
        rp_cf <- return_period(mdl, value = ev, covariate = cov_cf)
        rl_cf <- unname(return_level(mdl, rp = rp, covariate = cov_cf))

        c(pars,
          "disp" = disp, 
          "event_magnitude" = ev, 
          "return_period" = rp, 
          "PR" = rp_cf / rp, 
          "dI_abs" = ev - rl_cf, 
          "dI_rel" = (ev - rl_cf) / rl_cf * 100)
    } else {
        return(rep(NA, 10))
    }
}

In [3]:
# wrapper function to get bootstrapped confidence intervals for spreadsheet
boot_ci <- function(mdl, cov, cov_cf, ev = NA, rp = NA, seed = 42, nsamp = 500, dp = 5) {
    
    # get best estimate from the observed data
    event_value <- mdl$x[length(mdl$x)]
    mdl_res <- mdl_ests(mdl, cov, cov_cf, event_value, rp = rp)
    mdl_df <- mdl$cov.data
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        boot_mdl <- fevd_fixeddisp("rx21day", "gmst", boot_df, solver = "Nelder-Mead")
        mdl_ests(boot_mdl, cov, cov_cf, event_value, rp = rp)
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    return(boot_qq)
}

---
## **Trend analysis - observations**

In [14]:
gmst <- load_ts("ts/Italy-floods_gmst-smoothed.dat", col.names = c("year", "gmst"))
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_cf <- gmst_2023 - 1.2

In [16]:
# MSWEP
df <- merge(gmst, load_ts("ts/Italy-floods_rx21day-amj_mswep.dat", col.names = c("year", "rx21day")))
ns_mdl <- fevd_fixeddisp("rx21day", "gmst", df, solver = "Nelder-Mead")
boot_ci(ns_mdl, gmst_2023, gmst_cf, nsamp = 1000)

,bestimate,2.5%,97.5%
mu0,96.18517,82.11267,117.86247
sigma0,21.28290,14.07613,27.63517
shape,0.09425,-0.10335,0.49210
alpha,-2.43542,-37.65285,23.13395
dispersion,0.22127,0.14658,0.26690
event_magnitude,212.30070,212.30070,212.30070
return_period,95.15533,20.37165,30607.94235
PR,0.81911,0.00666,18.64574
dI_abs,-6.54958,-98.72365,61.11476
dI_rel,-2.99272,-31.74145,40.42360


In [18]:
# ERA5
df <- merge(gmst, load_ts("ts/Italy-floods_rx21day-amj_era5.dat", col.names = c("year", "rx21day")))
ns_mdl <- fevd_fixeddisp("rx21day", "gmst", df, solver = "Nelder-Mead")
boot_ci(ns_mdl, gmst_2023, gmst_cf, nsamp = 1000)

,bestimate,2.5%,97.5%
mu0,107.01738,97.51432,1.167080e+02
sigma0,27.69835,22.57624,3.218326e+01
shape,0.06337,-0.07344,2.366000e-01
alpha,-10.19596,-33.07732,1.246151e+01
dispersion,0.25882,0.21354,2.981700e-01
event_magnitude,346.58893,346.58893,3.465889e+02
return_period,1981.90924,133.31696,1.678320e+08
PR,0.39223,0.00241,2.603050e+00
dI_abs,-41.97905,-142.13912,4.875570e+01
dI_rel,-10.80353,-29.08348,1.637216e+01


## **Trend analysis - CORDEX**

In [4]:
model_results <- function(df, rp = 10, cov_pres, cov_pi, cov_fut, nsamp = 5, seed = 42) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    # fit models
    mdl_eval <- fevd_fixeddisp("rx21day", "gmst", df[df$year >= 1979 & df$year <= 2023,], solver = "Nelder-Mead")
    mdl_attr <- fevd_fixeddisp("rx21day", "gmst", df[df$year <= 2023,], solver = "Nelder-Mead")
    mdl_proj <- fevd_fixeddisp("rx21day", "gmst", df[df$year <= 2050,], solver = "Nelder-Mead")
    
    # get return level to use for analysis
    event_rl <- unname(return_level(mdl_attr, rp = rp, covariate = gmst_2023))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## Bootstrap each set of model results

    ci_eval <- boot_ci(mdl_eval, cov = cov_pres, cov_cf = cov_pi, ev = event_rl, rp = rp, nsamp = nsamp)[c("disp", "shape"),]
    ci_attr <- boot_ci(mdl_attr, cov = cov_pres, cov_cf = cov_pi, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", "dI_rel"),]
    ci_proj <- boot_ci(mdl_proj, cov = cov_pres, cov_cf = cov_fut, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", "dI_rel"),]
                   
    # invert future projections
    ci_proj["PR",] <- 1/ci_proj["PR",c(1,3,2)]
    ci_proj["dI_rel",] <- -ci_proj["dI_rel", c(1,3,2)]
    
    # flatten & rename
    ci_eval <- unlist(lapply(rownames(ci_eval), function(cnm) setNames(ci_eval[cnm,], paste("eval", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_attr <- unlist(lapply(rownames(ci_attr), function(cnm) setNames(ci_attr[cnm,], paste("attr", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_proj <- unlist(lapply(rownames(ci_proj), function(cnm) setNames(ci_proj[cnm,], paste("proj", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
                         
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    t(data.frame(c(ci_eval, "rp_value" = event_rl, ci_attr, ci_proj)))     
}

In [5]:
gmst <- load_ts("ts/Italy-floods_gmst-smoothed.dat", col.names = c("year", "gmst"))
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_pi <- gmst_2023 - 1.2
gmst_fut <- gmst_2023 + 1.2

# parameters for confidence intervals
rp <- 100
nsamp <- 1000

In [ ]:
# loop over all available models, get results for each
res <- do.call("rbind", sapply(list.files("ts", pattern = "rx21day-amj_EUR-11", full.names = T), function(fnm) {

    print(fnm)
    
    # load data   
    gmst_fnm <- list.files("ts", paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4], "_rcp85_", strsplit(fnm, "_")[[1]][5], ".dat"), full.names = T)
    df <- merge(load_ts(gmst_fnm, col.names = c("year", "gmst")), 
                load_ts(fnm, col.names = c("year", "rx21day")))
    
    res_df <- model_results(df, rp = rp, gmst_2023, gmst_pi, gmst_fut, nsamp = nsamp)
    rownames(res_df) <- gsub(".dat", "", paste0(strsplit(fnm, "_")[[1]][4:6], collapse = "_"))
    res_df
}, simplify = F))
write.csv(res, "Italy-floods_cordex-results.csv")